In [1]:
import numpy as np
import pandas as pd
import trackpy
import os
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime
import mspt.particle_fitting as fit
import mspt.image_processing as img
import mspt.particle_detection as detect
import mspt.trajectory_analysis1 as traj
import mspt.plotting as plot
os.environ['NUMEXPR_MAX_THREADS'] = '32'


pytorch/CUDA not available or not correctly installed


In [2]:
# General parameters
batch_mode = True
frame_range = []
navg = 1
mode = 'continuous_median'
window_length = 1001
parallel = True
GPU = False
thresh = 0.00055
DoG_estimates = {'T': 0.1423, 's': 2.1436, 'sigma': 1.2921}
dmax = 4.
max_frames_to_vanish = 0
minimum_trajectory_length = 5

# Movie acquisition parameters
frame_rate = 199.8
pixel_size = 84.4
n_timelags_MSD = None
n_timelags_JDD = None
slope = 32876.71
offset = -9.26


In [3]:
# Specify directory
directory = img.directoryDialog(os.getcwd())
filepaths_data = img.find_filepaths(directory, extension='mp', exclude=None)

assert len(filepaths_data) > 0, 'Cannot find any movie files to process in current directory'

# Process first file
filename = filepaths_data[0]
print(f"Selected file: {filename}")


Selected file: /Users/jamesmclean/Downloads/001_sample.mp


In [4]:
# Apply continuous median background removal
frames, file = img.mp_reader(batch_mode=batch_mode,
                             file_to_load=filename,
                             frame_range=frame_range,
                             mode=mode,
                             navg=navg,
                             window_length=window_length,
                             parallel=parallel,
                             GPU=GPU)
print("Background removal completed.")


Loaded /Users/jamesmclean/Downloads/001_sample.mp


Generating frames...:   0%|          | 0/75000 [00:00<?, ?frames/s]

Background removal completed.


In [5]:
# Detect and fit candidate spots
fitted_particles = fit.particle_fitter(frames,
                                       halfsize=window_length//2,
                                       thresh=thresh,
                                       method='trust-ncg',
                                       DoG_estimates=DoG_estimates)
print("Candidate spots detection and fitting completed.")


Identifying particle candidates...:   0%|          | 0/74000 [00:00<?, ?frames/s]

3196326 particle candidates identified


Fitting particles...:   0%|          | 0/3196326 [00:00<?, ?candidate spots/s]

Candidate spots detection and fitting completed.


In [ ]:
# Link trajectories
linked_trajectories = trackpy.link_df(fitted_particles, search_range=dmax, memory=max_frames_to_vanish)
linked_trajectories = linked_trajectories.sort_values(by=['particle', 'frame'])
trajectories_lenfilt = trackpy.filter_stubs(linked_trajectories, minimum_trajectory_length)
print("Trajectories linked and filtered.")


Frame 54084: 29 trajectories present.


In [ ]:
# Save trajectories to a temporary CSV file
trajectories_file = 'trajectories_temp.csv'
trajectories_lenfilt.to_csv(trajectories_file, index=False)
print(f"Trajectories saved to {trajectories_file}.")


In [ ]:
# Create a dummy output file path
dummy_output_file = 'dummy_output.h5'

# Fit trajectories and store results in memory
traj.fit_trajectories([trajectories_file],
                      output_file=dummy_output_file,
                      frame_rate=frame_rate,
                      pixel_size=pixel_size,
                      n_timelags_MSD=n_timelags_MSD,
                      n_timelags_JDD=n_timelags_JDD)
print("Trajectories fitting completed.")


In [ ]:
# Load data from the dummy output file
with pd.HDFStore(dummy_output_file, 'r') as dfs:
    results = pd.concat([dfs[key] for key in dfs.keys()])

# Clean up the dummy output file
os.remove(dummy_output_file)
os.remove(trajectories_file)
print("Data loaded from dummy output file and cleaned up.")


In [ ]:
# Convert contrast to mass using the linear relationship obtained from calibration measurements
results['median_mass'] = traj.apply_calibration(results, slope=slope, offset=offset)
print("Converted contrast to mass.")


In [ ]:
# Calculate the number of particles present for each trajectory 'particle number (linked)'
results['particle number (linked)'] = traj.calc_particle_number_linked(results)
results['particle number (detected)'] = traj.calc_particle_number_detected(results, '')
print("Calculated particle numbers.")


In [ ]:
# Plot data
fig, axs, df_kde, dat = plot.generate_2D_KDE(results,
                                             x='median_mass',
                                             y='Deff_MSD_JDD',
                                             x_range=(0, 400),
                                             y_range=(-1, 1),  # in log space
                                             figsize=(5, 5),
                                             traj_length=5,
                                             density=None,
                                             n_levels=12,
                                             cmap=mpl.cm.gray_r,
                                             alpha=1.0,
                                             show=True)
plt.show()
print("Plot generated.")
